In [1]:
from elasticsearch import Elasticsearch

In [2]:
ES_URL = "http://elasticsearch:9200"
INDEX = "data_playground_products"

In [3]:
es = Elasticsearch(hosts=[ES_URL])

In [4]:
response = es.count(
    index=INDEX,
    body={
        "query": {"match_all": {}},
    },
)
print(response["count"])

30000


In [5]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match_all": {}},
        "size": 3,
    },
)
print(
    *(r["_source"]["pid"] for r in response["hits"]["hits"]),
    sep="\n",
)

TSHFPS9EHSSHUWVU
SHTFPS228JPGPGHH
TSHFESTUMGXZCJGZ


In [6]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "casual"}},
        "size": 3,
    },
)
print(
    *(r["_source"]["title"] for r in response["hits"]["hits"]),
    sep="\n",
)

Men Brown Casual Sandal
Men Tan Casual Sandal
Men Beige Casual Sandal


In [7]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"description": "goku"}},
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!


In [8]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "multi_match": {
                "query": "ball",
                "fields": ["title", "description"],
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["title"], r["_source"]["description"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('Base Ball Cap Cap', 'Tee Buddy Caps are Comfortable fit for both men and women, Suitable for wearing at all occasions, Fabric is pre-washed before stitching. No color Fade for a long time, Total Length - 25 cm, Hood Width - 15 cm, Height - 13 cm')
('JAGUAR Football Shoes For Men\xa0\xa0(Black, Orange)', 'VECTOR-X \xa0JAGUAR Soccer Cleat provides exceptional ball touch and a comfortable,secure fit that promotes acceleration and quick directional changes on multiple kinds of surfaces , Dynamic Fit collar for secure comfort. Synthetic and textile upper for durable play.')
('Printed Men Round Neck Black T-Shirt', 'Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!')


In [9]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "must": [
                    {"range": {"average_rating": {"gt": 3}}},
                    {"match": {"category": "Clothing"}},
                ],
                "must_not": {
                    "range": {"discount": {"lte": 75}},
                },
                "should": {
                    "match": {"brand": "Dex"},
                },
            },
        },
        "size": 3,
    },
)
print(
    *(
        (
            r["_source"]["brand"],
            r["_source"]["average_rating"],
            r["_source"]["category"],
            r["_source"]["discount"],
        )
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('Amp', 3.4, 'Clothing and Accessories', 76.0)
('Amp', 3.4, 'Clothing and Accessories', 76.0)
('Amp', 3.4, 'Clothing and Accessories', 80.0)


In [10]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "term": {"description": "goku"},
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["description"] for r in response["hits"]["hits"]),
    sep="\n",
)

Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!


In [11]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "term": {"description": "gok"},
        },
        "size": 3,
    },
)
out = [r["_source"]["description"] for r in response["hits"]["hits"]] or ["None"]
print(
    *out,
    sep="\n",
)

None


In [12]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "range": {"selling_price": {"lt": 100}},
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["selling_price"] for r in response["hits"]["hits"]),
    sep="\n",
)

99.0
99.0


In [13]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "bool": {
                "filter": [
                    {"term": {"out_of_stock": False}},
                ],
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["pid"], r["_source"]["out_of_stock"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('TSHFPS9EHSSHUWVU', False)
('SHTFPS228JPGPGHH', False)
('TSHFESTUMGXZCJGZ', False)


In [14]:
response = es.search(
    index=INDEX,
    body={
        "aggs": {
            "categories": {
                "terms": {"field": "category.keyword"},
            },
        },
    },
)
print(response["aggregations"])

{'categories': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'Clothing and Accessories', 'doc_count': 28971}, {'key': 'Footwear', 'doc_count': 987}, {'key': 'Bags, Wallets & Belts', 'doc_count': 41}, {'key': 'Toys', 'doc_count': 1}]}}


In [15]:
response = es.search(
    index=INDEX,
    body={
        "sort": {"average_rating": {"order": "desc"}},
        "query": {"match_all": {}},
        "size": 3,
    },
)
print(
    *(r["_source"]["average_rating"] for r in response["hits"]["hits"]),
    sep="\n",
)

5.0
5.0
5.0


In [16]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match_all": {}},
        "from": 0,
        "size": 5,
    },
)
print(
    *(r["_source"]["pid"] for r in response["hits"]["hits"]),
    sep="\n",
)

TSHFPS9EHSSHUWVU
SHTFPS228JPGPGHH
TSHFESTUMGXZCJGZ
TSHFQ4FWQ42YYX9F
TSHFSYF7N8YZTGN4


In [17]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match_all": {}},
        "from": 2,
        "size": 3,
    },
)
print(
    *(r["_source"]["pid"] for r in response["hits"]["hits"]),
    sep="\n",
)

TSHFESTUMGXZCJGZ
TSHFQ4FWQ42YYX9F
TSHFSYF7N8YZTGN4


In [18]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "shirt"}},
        "highlight": {
            "fields": {"title": {}},
        },
        "size": 1,
    },
)
print(
    *(r["highlight"] for r in response["hits"]["hits"]),
    sep="\n",
)

{'title': ['Men T-<em>Shirt</em>']}


In [19]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "shirt"}},
        "highlight": {
            "tags_schema": "styled",
            "fields": {"title": {}},
        },
        "size": 1,
    },
)
print(
    *(r["highlight"] for r in response["hits"]["hits"]),
    sep="\n",
)

{'title': ['Men T-<em class="hlt1">Shirt</em>']}


In [20]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "shirt"}},
        "highlight": {
            "pre_tags": ["<tag1>"],
            "post_tags": ["</tag1>"],
            "fields": {"title": {}},
        },
        "size": 1,
    },
)
print(
    *(r["highlight"] for r in response["hits"]["hits"]),
    sep="\n",
)

{'title': ['Men T-<tag1>Shirt</tag1>']}


In [21]:
response = es.search(
    index=INDEX,
    body={
        "query": {"match": {"title": "shirt"}},
        "highlight": {
            "type": "plain",
            "fields": {"title": {}},
        },
        "size": 1,
    },
)
print(
    *(r["highlight"] for r in response["hits"]["hits"]),
    sep="\n",
)

{'title': ['Men T-<em>Shirt</em>']}


In [22]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "filter": [
                            {"term": {"out_of_stock": False}},
                        ],
                    },
                },
                "script": {
                    "source": "doc['average_rating'].size() == 0 || "
                    "doc['selling_price'].size() == 0 ? "
                    "0 : "
                    "doc['selling_price'].value / "
                    "(doc['average_rating'].value)",
                },
            },
        },
        "size": 3,
    },
)
print(
    *(
        (
            r["_source"]["out_of_stock"],
            r["_source"]["average_rating"],
            r["_source"]["selling_price"],
            r["_source"]["selling_price"] / r["_source"]["average_rating"],
        )
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

(False, 2.6, 7998.0, 3076.153846153846)
(False, 2.6, 6998.0, 2691.5384615384614)
(False, 2.6, 5998.0, 2306.9230769230767)


In [23]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "more_like_this": {
                "fields": ["title", "description"],
                "like": "dark pants",
                "min_term_freq": 1,
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["title"], r["_source"]["description"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('Solid Men Dark Blue Track Pants', "Men's Navy Stripped Track Pant")
('Solid Men Dark Blue Track Pants', 'A2Z Quality Group - Joggers / Track pants have been developed for those who like a form fitting pant with a full stretch! it is higher elastic fabric, they retain the same look as our normal training pants yet possess the ability to stretch and rebound much further. If you squat or train a lot in pants, these ones are for you! , and Training with High Quality Fabric Advanced Stretch fit, Elastic in waist band to give a comfortable grip. Please check the measurement before you buy the item.')
('Printed Men Dark Green Track Pants', '')


In [24]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "more_like_this": {
                "fields": ["title", "description"],
                "like": "slim shirt",
                "min_term_freq": 1,
            },
        },
        "size": 3,
    },
)
print(
    *(
        (r["_source"]["title"], r["_source"]["description"])
        for r in response["hits"]["hits"]
    ),
    sep="\n",
)

('Men Slim Fit Printed Slim Collar Casual Shirt', '')
('Men Slim Fit Printed Slim Collar Casual Shirt', '')
('Men Slim Fit Solid Slim Collar Casual Shirt', '')


In [25]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "exists": {"field": "product_details.Country of Origin"},
        },
        "aggs": {
            "unique_countries": {
                "terms": {"field": "product_details.Country of Origin.keyword"},
            },
        },
        "size": 3,
    },
)
print(
    *(r["key"] for r in response["aggregations"]["unique_countries"]["buckets"]),
    sep="\n",
)

India
China
Namibia
Bangladesh
Sri Lanka
Hong Kong
Vietnam
Cambodia
Turkey
Madagascar


In [26]:
response = es.search(
    index=INDEX,
    body={
        "query": {
            "prefix": {
                "seller": {"value": "ac"},
            },
        },
        "size": 3,
    },
)
print(
    *(r["_source"]["seller"] for r in response["hits"]["hits"]),
    sep="\n",
)

STYLE ACCORD
STYLE ACCORD
STYLE ACCORD
